In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers huggingface_hub
from huggingface_hub import login

# Replace 'YOUR_HUGGINGFACE_TOKEN' with your actual token
token = 'you_token'
login(token=token)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install datasets transformers

from datasets import load_dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, TrainingArguments, Trainer
import torch

In [ ]:
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Select a subset of the dataset for quicker training
train_size = 500  # Number of training samples
val_size = 100    # Number of validation samples
small_train_dataset = dataset['train'].shuffle(seed=42).select(range(train_size))
small_val_dataset = dataset['validation'].shuffle(seed=42).select(range(val_size))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenize the dataset
def preprocess_data(examples):
    inputs = [doc for doc in examples['article']]
    targets = [doc for doc in examples['highlights']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=150, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Map the preprocessing function to the dataset
tokenized_train_dataset = small_train_dataset.map(preprocess_data, batched=True, remove_columns=["article", "highlights", "id"])
tokenized_val_dataset = small_val_dataset.map(preprocess_data, batched=True, remove_columns=["article", "highlights", "id"])


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,  # Reduced number of epochs for quicker training
    per_device_train_batch_size=2,  # Adjust batch size based on available memory
    gradient_accumulation_steps=8,  # Accumulate gradients to simulate a larger batch size
    evaluation_strategy="steps",
    save_steps=1000,
    eval_steps=500,  # Evaluate more frequently to monitor progress
    logging_steps=250,  # Log more frequently to monitor training
    save_total_limit=2,
    fp16=False,  # Mixed precision training is not supported on CPU
    load_best_model_at_end=True,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()


Step,Training Loss,Validation Loss


TrainOutput(global_step=31, training_loss=3.581505560105847, metrics={'train_runtime': 5243.2393, 'train_samples_per_second': 0.095, 'train_steps_per_second': 0.006, 'total_flos': 302043113717760.0, 'train_loss': 3.581505560105847, 'epoch': 0.992})

In [ ]:
# Save the model
model.save_pretrained("/content/drive/MyDrive/fine_tuned_model")
tokenizer.save_pretrained("/content/drive/MyDrive/fine_tuned_model")

('/content/drive/MyDrive/fine_tuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_model/spiece.model',
 '/content/drive/MyDrive/fine_tuned_model/added_tokens.json')